<a href="https://colab.research.google.com/github/dtht2d/bispectrump_component/blob/main/codes/compute_bispectrum_component.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compute Bispectrum Components
---
## Outline
**Computational steps**

**1. Data prep and parameter**

**a. Data prep**

- Read CIF file as data frame

**b. Define position of neighbor atoms $k$ relative to a central atom $i$  is a point within the 3D ball of radius**

- Estimate list of potentially atoms in the center cell
- Choose a center atom $i$  from the list and get its coordinate $(x_i,y_i,z_i)$
- Re-calculate coordinate for all atoms in the cell w.r.t new reference of frame  (origin at $(x_i,y_i,z_i)$)
- Compute neighbors list in a chosen cutoff radius $R_{cut}$ with respect to new reference frame where its origin is at center atom $i$  location

**c. Map possible neighbor atoms on to the set of points $(\theta_0, \theta,\phi)$**

Compute $\theta_0, \ \theta, \ \phi$

**2. Compute bispectrum component for input values $(j_1, j_2, j)$**

1. Compute Clebsch- Gordan coefficient

    $$
    C^{jm}_{{j_1}{m_1}{j_2}{m_2}}, C^{jm'}_{{j_1}{m'_1}{j_2}{m'_2}}
    $$

2. Compute coupling coefficient

    $$
     H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}
    $$

3. Compute  density coefficient

    $$
    u^j_{mm'}, \mu_{m_1m_1'}^{j_1}, \mu_{m_2,m_2'}^{j_2}
    $$
---

## Install packages


In [ ]:
pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install sympy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
biopython                     1.80
bleach                        5.0.1
blis                          0.7.9
bokeh  

In [ ]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy.physics.quantum.cg import CG
from sympy.physics.wigner import wigner_d
from sympy.physics.quantum.spin import Rotation
from sympy import *
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
from methods
import itertools

SyntaxError: ignored

## 1. Data prep and parameter
### a. Data prep

**→ set of data, neighbor list, it could be x, y, z coordinate and convert to r, $\theta_0,\theta,\phi$**

- read CIF file as data frame

    [avgBL-Model.cif](Compute%20Bispectrum%20Components%20fe15e8c590c94d438538084914489ecb/avgBL-Model.cif)

- cell length (unit angstrom)
- $R_{cut }$ cut off radius

    ********Note:******** Chosen $R_{cut}$ needs to divide by the true cell length (i.e: with example dataset we need to divide by 18.7337) since $(x,y,z)$  are fraction with cell dimension (1,1,1), $R_i, R_k$ (center, neighbor): atom radius, measures from center of nucleus to the outermost isolated electron also so need to divide by

- $r_\mu$  atomic radius for all chemical elements
- $x, y, z$ coordinate for all atoms
- atom type

In [ ]:
path = "/content/avgBL-Model.cif"
dico = MMCIF2Dict(path)
df_cif = pd.DataFrame.from_dict(dico, orient='index')
print (df_cif)

NameError: ignored

**Comment:**
Define index as orientation, column label in CIF file now become row,  index 0 column store all value atom coordinate values that need to unpack and assigned as new numpy array to create new data frame that column name is X, Y, Z coordinates and row are coordinate values for each atom in the cell
    **Note:** $x, y, z$  are fraction of cell length to 1. True value needs to multiply with actual cell length

In [ ]:
path = "/Users/DuongHoang/UMKC-Grad/UMKC_Research/bispectrump_component/data/avgBL-Model.cif"
dico = MMCIF2Dict(path)
df_cif = pd.DataFrame.from_dict(dico, orient='index')
x = df_cif.iloc[-3]
y = df_cif.iloc[-2]
z = df_cif.iloc[-1]
atom_type = df_cif.iloc[-4]
x_array = np.array(x[0],dtype=float)
y_array = np.array(y[0],dtype=float)
z_array = np.array(z[0],dtype=float)
atom_type_array = np.array(atom_type[0], dtype=str)
df = pd.DataFrame({"atom_type":atom_type_array,"X" : x_array, "Y":y_array, "Z": z_array})
print(df)

    atom_type         X         Y         Z
0          si  0.954659  0.013642  0.469779
1          si  0.723469  0.727441  0.237191
2          si  0.911910  0.842907  0.722894
3          si  0.180034  0.242715  0.263001
4          si  0.019245  0.647721  0.350679
..        ...       ...       ...       ...
335        si  0.719519  0.348038  0.504628
336        si  0.370296  0.945794  0.229344
337        si  0.206532  0.594928  0.133419
338        si  0.743145  0.839250  0.191284
339        si  0.018592  0.043251  0.767526

[340 rows x 4 columns]


Date frame with atoms $ (x,y,z) $  coordinates and atom type

### b. **Define position of a neighbor atom $k$ relative to a central atom $i$  is a point within the 3D ball of radius**

**Idea :** choose a central atom  $i$ where its coordinate $(x_i,y_i,z_i)$ in the center of a single cell (reference coordinate frame, origin point $O(0,0,0)$ and orthogonal axes $X,Y,Z$). Re-calculate coordinate for all atoms in the cell by shift origin to $(x_i,y_i,z_i)$.

Coordinate of neighbor atom with respect to the new system is:

$$
\begin{bmatrix}x_k \\ y_k \\ z_k \end{bmatrix} = \begin{bmatrix} x - x_i \\ y-y_i \\ z-z_i \end{bmatrix}
$$
**Estimate list of potentially atoms in the center cell**

**Condition:** $\text{range}_{min} + R_{cut}, \text{range}_{max}+R_{cut} \leq$  1

**Example:**

$$
0.5\leq x,y,z \leq 0.7
$$


In [ ]:
df_atoms = df[(df['X'].between(0.5,0.7,inclusive='both'))
                         & (df['Y'].between(0.5,0.7,inclusive='both'))
                         & (df['Z'].between(0.5,0.7, inclusive='both'))]
print (df_atoms)

    atom_type         X         Y         Z
17         si  0.544332  0.665575  0.559883
152        si  0.696335  0.535755  0.620077
311        si  0.652860  0.654536  0.608059


**Note**: Choose atom ID 17 as center atom, in df_atoms its index  row is [0] but in df_new its index row is [17].

In [ ]:
atom_i =df.iloc[17]
x_i = df['X'].iloc[17]
y_i = df['Y'].iloc[17]
z_i = df['Z'].iloc[17]
print(x_i,y_i,z_i)

0.544332 0.665575 0.559883


Re-calculate coordinate for all atoms (w.r.t new reference of frame  (origin at $(x_i,y_i,z_i)$), and distance $r_{ik}$  from a center atom $i$  to other atom in the cell= np.sqrt(np.square(X_k_array)+np.square(Y_k_array)+np.square(Z_k_array))

In [ ]:
X_array = df['X'].to_numpy()
Y_array = df['Y'].to_numpy()
Z_array = df['Z'].to_numpy()
X_k_array = X_array - x_i
Y_k_array = Y_array - y_i
Z_k_array = Z_array - z_i
r_ik= np.sqrt(np.square(X_k_array)+np.square(Y_k_array)+np.square(Z_k_array))
df['X_k'],df['Y_k'], df['Z_k'],df['r_ik']= X_k_array, Y_k_array, Z_k_array, r_ik
print(df)

    atom_type         X         Y         Z       X_k       Y_k       Z_k  \
0          si  0.954659  0.013642  0.469779  0.410327 -0.651933 -0.090104   
1          si  0.723469  0.727441  0.237191  0.179137  0.061866 -0.322692   
2          si  0.911910  0.842907  0.722894  0.367578  0.177332  0.163011   
3          si  0.180034  0.242715  0.263001 -0.364298 -0.422860 -0.296882   
4          si  0.019245  0.647721  0.350679 -0.525087 -0.017854 -0.209204   
..        ...       ...       ...       ...       ...       ...       ...   
335        si  0.719519  0.348038  0.504628  0.175187 -0.317537 -0.055255   
336        si  0.370296  0.945794  0.229344 -0.174036  0.280219 -0.330539   
337        si  0.206532  0.594928  0.133419 -0.337800 -0.070647 -0.426464   
338        si  0.743145  0.839250  0.191284  0.198813  0.173675 -0.368599   
339        si  0.018592  0.043251  0.767526 -0.525740 -0.622324  0.207643   

         r_ik  
0    0.775567  
1    0.374229  
2    0.439469  
3    0.6321

Check to see if chosen center atom coordinate sets to (0,0,0)

In [ ]:
print(df.iloc[17])

atom_type          si
X            0.544332
Y            0.665575
Z            0.559883
X_k               0.0
Y_k               0.0
Z_k               0.0
r_ik              0.0
Name: 17, dtype: object


**Compute neighbors list in a chosen cutoff radius $R_{cut}$ with respect to new reference frame where its origin is at center atom $i$  location**

$$
R_{cut}= some \ reasonable \ value
$$

**Example:** Sillicon atomic radius is $1.46\text{\AA}$ , Silicon–silicon π single bond ****$2.853 \text{\AA}$**, $R_{cut}= \frac{1.46+1.46}{cell\ length}=\frac{2.92}{18.7337} \approx 0.16$ but let choose $R_{cut} =0.25$

$$
r_{ik}= \sqrt{x_k^2 +y_k^2+z_k^2}
$$

Assume atom of element $ \mu $ (in this case it’s Silicon) is a sphere with radius $r_\mu$. We want to make sure that for the neighbors list only atoms whose entire sphere is in within the bigger sphere with radius $R_{cut}$.

$$
r_\mu = \frac{atomic\ radius \ (\text{\AA})}{cell \ length \ (\text{\AA})} = \frac{1.46}{18.7337} \approx 0.0779
$$

**Condition:**

$$
r_{ik}+r_\mu\leq R_{cut}
$$

In [ ]:
#INPUT values
atomic_radius = 1.46            #silicon atomic radius, unit: angstrom
cell_length = df.iloc[4]        #index row start from 0 _cell_length_a at row 5 index [4]

r_mu = 0.0779                   #scale atomic radius w.r.t cell length
R_cut = 0.25                    #scaled value w.r.t cell length (for Si-Si case)
df_ik = df[(df['r_ik'] + r_mu)<= (R_cut)].copy(deep=true)
print(df_ik[['X_k', 'Y_k', 'Z_k', 'r_ik']])

          X_k       Y_k       Z_k      r_ik
17   0.000000  0.000000  0.000000  0.000000
35  -0.103248 -0.005137 -0.122932  0.160620
72   0.014310  0.043550 -0.114092  0.122957
80  -0.072758  0.086734  0.045996  0.122197
128 -0.050784 -0.108715  0.039094  0.126199
311  0.108528 -0.011039  0.048176  0.119252



## c. Map possible neighbor atoms on to the set of points $(\theta_0, \theta,\phi)$

Ref.[3], eq.(3), p. 2

**Idea:** project the atomic density onto the surface of four- dimensional unit sphere, similarly to how the Riemann sphere is constructed with the transformation

$$
(\theta_0, \theta,\phi) =[|r|/r_0, cos^{-1}(|z'|/|r|), tan^{-1}(y'/x'))]
$$

$r_0>R_{cut}/\pi$ → so that the 4D surface contains all the information from 3D spherical region inside the cutoff, include radial dimension

**$\text{rotation angel}\ \omega \equiv \theta_0 \ \text{about some axis} \ n(\Theta \equiv \theta, \Phi \equiv \phi)$**

**Condition:**

Ref.[5] session 1.4.2, p.23

(1) $0 \leq \theta_0 \leq \pi$

(2) $0\leq \theta \leq \pi$

(3) $0 \leq \phi \leq 2\pi$.

**Note**:  $ \phi = tan^{-1}(\frac{y'}{x'})$, $x', y'$  has negative values → need to convert range of angle from $[-\pi, \pi] \ \text{to}\ [0, 2\pi] $

(4) $R_{cut}$ depends on the chemical identities of both the neighbor atom and center atom (at a distance that is less force affected on both of them?)
Compute $\theta_0, \ \theta, \ \phi$

$$
\theta_{0} = \theta_0^{max}\frac{|r_{ik}|}{|r_0|}=\pi\frac{\sqrt{x_k^2+y^2_k+z_k^2}}{R_{cut}}
$$

where $\theta_0^{max}= \pi$

->excluded points south  of latitude $\theta_0=r_{frac0}\pi$

In [ ]:
#theta_0
r_ik_array = df_ik['r_ik'].to_numpy() #r_ik from selected neighbors
r_0_array = np.full((r_ik_array.shape),R_cut)
theta_0_array = np.pi*(np.divide(r_ik_array,r_0_array))
print (r_0_array)
#theta
Z_k_abs_array = np.abs(df_ik['Z_k'].to_numpy())
theta_array = np.arccos(np.divide(Z_k_abs_array,r_ik_array))
#phi
X_k_array = df_ik['X_k'].to_numpy()
Y_k_array = df_ik['Y_k'].to_numpy()
phi_array = np.arctan(np.divide(Y_k_array, X_k_array))
#convert angle to positive value between [0,2pi]
phi_array_convert = np.mod(phi_array, 2*np.pi)
for angle in phi_array_convert:
    if (angle >=2*np.pi) and (angle < 0):
        raise ValueError('phi angle in between 0 and 2pi')
print(phi_array_convert)
print(phi_array)

[0.25 0.25 0.25 0.25 0.25 0.25]
[       nan 0.049713   1.25332279 5.41038219 1.13378905 6.18181825]
[        nan  0.049713    1.25332279 -0.87280312  1.13378905 -0.10136706]


/var/folders/7_/1phpw0g101sfbhw8vl2jpp5h0000gn/T/ipykernel_98758/3703012928.py:8: RuntimeWarning: invalid value encountered in divide
  theta_array = np.arccos(np.divide(Z_k_abs_array,r_ik_array))
/var/folders/7_/1phpw0g101sfbhw8vl2jpp5h0000gn/T/ipykernel_98758/3703012928.py:12: RuntimeWarning: invalid value encountered in divide
  phi_array = np.arctan(np.divide(Y_k_array, X_k_array))


Replace NaN with 0: (error for invalid value center atom values since we have 0/0)

In [ ]:
#Replace NaN with 0: (code will have error for invalid value center atom values 0/0)
df_ik['theta_0'] = theta_0_array
df_ik['theta_0'] = df_ik['theta_0'].replace(np.nan,0)
df_ik['theta'] = theta_array
df_ik['theta'] = df_ik['theta'].replace(np.nan,0)
df_ik['phi'] = phi_array_convert
df_ik['phi'] = df_ik['phi'].replace(np.nan,0)
print(df_ik.drop(['X', 'Y', 'Z', 'atom_type'], axis =1))


          X_k       Y_k       Z_k      r_ik   theta_0     theta       phi
17   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
35  -0.103248 -0.005137 -0.122932  0.160620  2.018412  0.699198  0.049713
72   0.014310  0.043550 -0.114092  0.122957  1.545120  0.382047  1.253323
80  -0.072758  0.086734  0.045996  0.122197  1.535575  1.184880  5.410382
128 -0.050784 -0.108715  0.039094  0.126199  1.585869  1.255835  1.133789
311  0.108528 -0.011039  0.048176  0.119252  1.498569  1.154929  6.181818


In [ ]:
j1,j2,j,m1,m2,m,m1p,m2p,mp = (3/2,1/2,1,3/2,-1/2,1,3/2,1/2,2)
H=getCoeffH(j1,j2,j,m1,m2,m,m1p,m2p,mp)
print(H)

0


### c. **Compute expansion coefficients density function $u^j_{m,m'}, \ u^{j_1}_{m_1,m_2}, \ u^{j_2}_{m_1,m_2}$**

**Expansion coefficient for the partial density of neighbors of element $\mu$**

**Definition**

$$
 u^j_{m,m'} = \langle U^j_{m'm} | \rho\rangle
$$

$$
u_{j,m,m'}^{\mu} = w^{self}_{\mu_i \mu} U_{j,m,m'}(0,0,0) + \displaystyle\sum_{r_{ik}<R_{cut}^{\mu_i\mu_k}} f_c(r_{ik};R_{cut}^{\mu_i\mu_k})w_{{\mu}_k}U_{m,m'}^{j}(\theta_{0},\theta, \phi) \delta_{\mu\mu_k}
$$

In [ ]:
#EXAMPLE
j,m,mp = 3,2,3
#array for weight coefficient w.r.t to atom type, since we deal with single type set it equal to 1
w_ik_arr = np.full((r_ik_array.shape),1)
#delta function delta=1 if i and k has the same element type, if not delta =0
delta = np.full((r_ik_array.shape),0)
delta_arr = np.where(df_ik['atom_type']==df_ik['atom_type'].iloc[0],1,delta)
print (w_ik_arr)
print (delta_arr)

[1 1 1 1 1 1]
[1 1 1 1 1 1]


In [ ]:
def getDensityFunction_u(j,m,mp,w_ik_array, delta_array,r_ik_array, r_min0, R_cut, theta_0_array, theta_array, phi_array):
        '''
        Args:
            j: angular momentum
            m: eigenvalue of angular momentum
            mp: eigenvalue of j along rotated axis
            w_ik_arr: array for the coefficients that are dimensionless weights that are chosen to distinguish atoms
                        of different types, while the central atom is arbitrarily assigned a unit weight, dimensin (1,k)
            delta_arr: array for the Dirac delta function, indicates only neighbor atom of element the same as center atom
                        contribute to partial density,  dimension (1,k)
            r_ik_array: array for distance from center atom to neighbor atom, dimension (1,k+1), k is number of neighbor atoms
                        in cutoff radius, array include center atom as well
            r_min0: number, parameter in distance to angle conversion (distance units), choose
            R_cut: number, cutoff radius
            theta_0_array: array for theta_0 angel (fist angle of rotation [0,pi])
                        of neighbor atoms in reference frame of center atom, dimension (1, k+1)
            theta_array: array for theta angel ( second angle of rotation [0,pi])
                        of neighbor atoms in reference frame of center atom, dimension (1, k+1)
            phi_array: array for phi angel (third angle of rotation [0,2pi])
                        of neighbor atoms in reference frame of center atom, dimension (1, k+1)
        Returns: expansion coefficients density function u_jm_mp
        '''
        R_cut_array = np.full((r_ik_array.shape), R_cut)
        f_cut_arr = (1 / 2) * (np.cos(np.pi * (np.divide(r_ik_array - r_min0, R_cut_array - r_min0))) + 1)
        U_jmmp = []
        for theta_0, theta, phi in zip(theta_0_array, theta_array, phi_array):
            mpp_list = np.linspace(-j, j, int(2 * j + 1))
            U = 0
            for mpp in mpp_list:
                rot1 = Rotation.D(j, m, mpp, phi, theta, - phi)
                rot2 = Rotation.D(j, mpp, mp, phi, -theta, -phi)
                Dj_m_mpp = rot1.doit()
                Dj_mpp_mp = rot2.doit()
                Um_mp = Dj_m_mpp * (exp(-I * mpp * theta_0)) * Dj_mpp_mp
                Um_mp = nsimplify(Um_mp)
                U += N(Um_mp)
            U_jmmp.append(U)
        U_jmmp_array = np.array(U_jmmp, dtype='complex')
        U_jmmp_arr = np.where(np.isnan(U_jmmp_array), 0, U_jmmp_array)
        u_jmmp = np.dot((f_cut_arr*U_jmmp_arr),(w_ik_arr*delta_arr))
        return u_jmmp

**Comment:**
Because  $U_{j,m,m'}(0,0,0) = 0$  so we ignore self contribution $w_{\mu_i\mu}^{self}$.
We simplify:
$$
u_{j,m,m'}^{\mu} = w^{self}_{\mu_i \mu} U_{j,m,m'}(0,0,0) + \displaystyle\sum_{r_{ik}<R_{cut}^{\mu_i\mu_k}} f_c(r_{ik};R_{cut}^{\mu_i\mu_k})w_{{\mu}_k}U_{m,m'}^{j}(\theta_{0},\theta, \phi) \delta_{\mu\mu_k}
$$
to
$$
u_{j,m,m'}^{\mu} =  \displaystyle\sum_{r_{ik}<R_{cut}^{\mu_i\mu_k}} f_c(r_{ik};R_{cut}^{\mu_i\mu_k})w_{{\mu}_{ik}}U_{m,m'}^{j}(\theta_{0},\theta, \phi) \delta_{\mu\mu_k}
$$

### Compute bispectrum components
$$
B_(j_1,j_2,j) = \displaystyle\sum_{{m,m'= -j}}^j  \left(u^{\smash{j}}_{m,m'}\right)^*\displaystyle\sum_{{m_1,m'_1 =-j_1}}^{j_1} \displaystyle\sum_{{m_2,m'_2 = -j_2}}^{j_2} H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}  u^{j_1}_{{m_1},{m_1'}} u^{j_2}_{{m_2},{m_2'}}
$$
- $C^{jm}_{{j_1}{m_1}{j_2}{m_2}} C^{jm'}_{{j_1}{m'_1}{j_2}{m'_2}} \equiv H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}$
- $\displaystyle\sum_{{m,m'= -j}}^j  \left(u^{\smash{j}}_{m,m'}\right)^*$
- $\displaystyle\sum_{{m_1,m'_1 =-j_1}}^{j_1} \displaystyle\sum_{{m_2,m'_2 = -j_2}}^{j_2} H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}  u^{j_1}_{{m_1},{m_1'}} u^{j_2}_{{m_2},{m_2'}}$

**Example**: $ j_1=1, j_2=2, j=3 $

In [ ]:
j = 3
j1 = 1
j2 = 2
m = np.linspace(-j, j, int(2 * j + 1)).tolist()
mp = np.linspace(-j, j, int(2 * j + 1)).tolist()
m1 = np.linspace(-j1, j1, int(2 * j1 + 1)).tolist()
m1p = np.linspace(-j1, j1, int(2 * j1 + 1)).tolist()
m2 = np.linspace(-j2, j2, int(2 * j2 + 1)).tolist()
m2p = np.linspace(-j2, j2, int(2 * j2 + 1)).tolist()
print(m1,m2,m,m1p,m2p,mp)

[-1.0, 0.0, 1.0] [-2.0, -1.0, 0.0, 1.0, 2.0] [-3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0] [-1.0, 0.0, 1.0] [-2.0, -1.0, 0.0, 1.0, 2.0] [-3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0]


In [ ]:
list_set = itertools.product(m1,m2,m,m1p,m2p,mp)
print(np.array(list_set))

In [ ]:
B_sum =[]
for i in list_set:
    m1, m2, m, m1p, m2p, mp = i
    H = getCoeffH(j1,j2,j,m1,m2,m,m1p,m2p,mp)
    if H==0:
        pass
    else:
        u_jmmp = getDensityFunction_u(j, m, mp, w_ik_arr, delta_arr, r_ik_array, 0,
                                      R_cut, theta_0_array, theta_array,phi_array)
        u1_j1m1m1p = getDensityFunction_u(j1, m1, m1p, w_ik_arr, delta_arr, r_ik_array, 0,
                                          R_cut, theta_0_array,theta_array, phi_array)
        u2_j2m2m2p = getDensityFunction_u(j2, m2, m2p, w_ik_arr, delta_arr, r_ik_array, 0,
                                          R_cut, theta_0_array,theta_array, phi_array)
        B_each = np.conj(u_jmmp) * H * (u1_j1m1m1p) * (u2_j2m2m2p)
        B_sum = N(B_each)
print(B_sum)

# References
---

[1] `Thompson, Swiler, Trott, Foiles, Tucker,` ***Spectral neighbor analysis method for automated generation of quantum-accurate interatomic potentials***  (2015) [https://www.osti.gov/biblio/1426894](https://www.osti.gov/biblio/1426894)

[2]  `J. K. Mason,` ***The relationship of the hyperspherical harmonics to SO(3), SO(4) and orientation distribution functions***, Acta Cryst A65, 259 (2009)

[https://libraryh3lp.com/file/8j46b06z956ega%40web.libraryh3lp.com/1665598079.pdf?t=6sfhoB1XgCcztCUZBmTEDs](https://libraryh3lp.com/file/8j46b06z956ega%40web.libraryh3lp.com/1665598079.pdf?t=6sfhoB1XgCcztCUZBmTEDs)

[3] *`A. Bartok, M. C. Payne, K. Risi, G. Csanyi,`**Gaussian approximation potentials: The accuracy of quantum mechanics, without the electrons*** (2010) [https://arxiv.org/pdf/0910.1019.pdf](https://arxiv.org/pdf/0910.1019.pdf)

[4] `A V Meremianin,*`  ***Multipole expansions in four-dimensional hyperspherical harmonics,*** J. Phys. A: Math. Gen. 39 3099 (2006) [https://iopscience.iop.org/article/10.1088/0305-4470/39/12/017/pdf?casa_token=YfzEUY2g4jwAAAAA:bqMuUwTRpQXDQEpCSvvwmlFYX6hi0xis-vCqLThjemvfDObHjP7XZw28oexUMra9FGg7AV1FVKvhtzZJn28g](https://iopscience.iop.org/article/10.1088/0305-4470/39/12/017/pdf?casa_token=YfzEUY2g4jwAAAAA:bqMuUwTRpQXDQEpCSvvwmlFYX6hi0xis-vCqLThjemvfDObHjP7XZw28oexUMra9FGg7AV1FVKvhtzZJn28g)

[5] *`D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,` ***Quantum Theory of Angular Momentum*** (1988) [https://library.oapen.org/handle/20.500.12657/50493](https://library.oapen.org/handle/20.500.12657/50493)

[6]  *`M. A. Cusentino,  M. A. Wood, A. P. Thompson,`***Explicit Multi-element Extension of the Spectral Neighbor Analysis Potential for Chemically Complex Systems,*** J. Phys. Chem. A,124, 26, 5456–5464 (2020) [https://doi.org/10.1021/acs.jpca.0c02450](https://doi.org/10.1021/acs.jpca.0c02450)

[7] `*LAMMPS,*` ***Compute sna/atom command,*** [https://docs.lammps.org/compute_sna_atom.html](https://docs.lammps.org/compute_sna_atom.html)

*[8] `Kyushin, S., Kurosaki, Y., Otsuka, K. et al.` ***Silicon–silicon π single bond*** Nat Commun* **11**, 4009 (2020). https://doi.org/10.1038/s41467-020-17815-z